# MultiMatest

## Libraries

In [1]:
import parser
import itertools
import pandas as pd

In [2]:
logic = parser.Parser(language=parser.propositional_language())
predicates = logic.language.predicates
operators = logic.language.ops
formula = "a&(a->~b)->a"
formula_parsed = logic.parse(formula)
values = list(range(2))
min_designated = 1
designated_values = list(filter(lambda x: x>=min_designated, values))

In [24]:
inputs = formula_parsed.inputs
formula_to_interpret = list(map(lambda x: x.decode('ascii'),formula_parsed.ops.copy()))

In [43]:
formula_to_interpret

['a', 'a', 'b', '~', '->', 'a', '->', '&']

In [4]:
preds = list(sorted(set(filter(lambda x: x in predicates, formula_to_interpret))))

In [14]:
interpretations = list(itertools.product(values, repeat = len(preds)))

In [35]:
interpretations

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [38]:
replacements = [dict(zip(preds, interpretation)) for interpretation in interpretations]

In [39]:
replacements

[{'a': 0, 'b': 0}, {'a': 0, 'b': 1}, {'a': 1, 'b': 0}, {'a': 1, 'b': 1}]

In [41]:
interp = list(map(replacements[0].get, formula_to_interpret, formula_to_interpret))

In [46]:
def unary_operator(matrix):
    return lambda x: matrix[x]

def binary_operator(matrix):
    return lambda x, y: matrix[x][y]

negation = unary_operator([1,0])

conjuntion = binary_operator([[0,0],
                              [0,1]])
disjunction = binary_operator([[0,1],
                               [1,1]])
implication = binary_operator([[1,1],
                               [0,1]])

logic_functions = {'~':negation, '&':conjuntion, 'v':disjunction, '->': implication}

In [11]:
interp

[0, 0, 0, '~', '->', 0, '->', '&']

In [30]:
inputs

[[], [], [], [-1], [-3, -1], [], [-2, -1], [-7, -1]]

In [16]:
pd.DataFrame([interp, inputs])

,0,1,2,3,4,5,6,7
0,0,0,0,~,->,0,->,&
1,[],[],[],[-1],"[-3, -1]",[],"[-2, -1]","[-7, -1]"


In [32]:
interp_func = list(map(logic_functions.get, interp, interp))

In [62]:
interpret_function = list(map(logic_functions.get, formula_to_interpret, formula_to_interpret))
formula_interpretations = [list(map(replacement.get, interpret_function, interpret_function)) for replacement in replacements]

In [87]:
formula_interpretations[0]

[0,
 0,
 0,
 <function __main__.unary_operator.<locals>.<lambda>(x)>,
 <function __main__.binary_operator.<locals>.<lambda>(x, y)>,
 0,
 <function __main__.binary_operator.<locals>.<lambda>(x, y)>,
 <function __main__.binary_operator.<locals>.<lambda>(x, y)>]

In [ ]:
def interpret_formula(interpretation, inputs):
    

In [85]:
new_interpretation = []
for formula_interpretation in formula_interpretations:
    
    interpretation_ = []

    for index, symbol in enumerate(formula_interpretation):
        if symbol in values:
            interpretation_.append(symbol)
        else:
            places = list(map(lambda x: index+x, inputs[index]))
            vals = list(map(lambda x: interpretation_[x], places))
            val = symbol(*vals)
            interpretation_.append(val)
            print(index, symbol, places, vals, symbol(*vals))

    new_interpretation.append(interpretation_)

3 <function unary_operator.<locals>.<lambda> at 0x7f8197fb43a0> [2] [0] 1
4 <function binary_operator.<locals>.<lambda> at 0x7f8197bf08b0> [1, 3] [0, 1] 1
6 <function binary_operator.<locals>.<lambda> at 0x7f8197bf08b0> [4, 5] [1, 0] 0
7 <function binary_operator.<locals>.<lambda> at 0x7f8197bf0040> [0, 6] [0, 0] 0
3 <function unary_operator.<locals>.<lambda> at 0x7f8197fb43a0> [2] [1] 0
4 <function binary_operator.<locals>.<lambda> at 0x7f8197bf08b0> [1, 3] [0, 0] 1
6 <function binary_operator.<locals>.<lambda> at 0x7f8197bf08b0> [4, 5] [1, 0] 0
7 <function binary_operator.<locals>.<lambda> at 0x7f8197bf0040> [0, 6] [0, 0] 0
3 <function unary_operator.<locals>.<lambda> at 0x7f8197fb43a0> [2] [0] 1
4 <function binary_operator.<locals>.<lambda> at 0x7f8197bf08b0> [1, 3] [1, 1] 1
6 <function binary_operator.<locals>.<lambda> at 0x7f8197bf08b0> [4, 5] [1, 1] 1
7 <function binary_operator.<locals>.<lambda> at 0x7f8197bf0040> [0, 6] [1, 1] 1
3 <function unary_operator.<locals>.<lambda> at 0

In [86]:
pd.DataFrame(new_interpretation, columns=formula_to_interpret,index=interpretations)

,a,a,b,~,->,a,->,&
"(0, 0)",0,0,0,1,1,0,0,0
"(0, 1)",0,0,1,0,1,0,0,0
"(1, 0)",1,1,0,1,1,1,1,1
"(1, 1)",1,1,1,0,0,1,1,1
